In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as stats
import re
import scipy.interpolate
from matplotlib.lines import Line2D
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['figure.figsize'] = (8, 6)
%config InlineBackend.figure_format = 'retina'

import sys
sys.path.append('pazy/sharpy-analysis-tools/')
import linear.stability as stability

# SHARPy DynamicCoupled sim with VF above

In [3]:
import os
sys.path.append('pazy/pazy_model/')
from generate_deformed_tseries import generate_pazy_tseries
import postprocess_deformed_flutter
controlled = False
flap_frac = 0.25
chord_frac = 0.25

# M = 16  # default 16
# N = 1  # default 1
# Ms = 20  # default 20
skin_on = False  # default False
trailing_edge_weight = False  # default False
alpha = 5
gravity_on = False
end_time = 0.1

M = 16
N = 1
Ms = 20

pazy_settings = {'num_elem': N,
                 'surface_m': M,
                'controlled': controlled,
                'cs_deflection': [0, 0],
                'cs_span_frac': flap_frac,
                'cs_chord_frac': chord_frac,
                'skin_on': 'off',
                'discretisation_method': 'michigan',
                }


from datetime import datetime
# u_inf_vec = np.linspace(10, 90, 81)
# u_inf_vec = [83]

output_folder = 'output/pazy_M{:g}N{:g}Ms{:g}_alpha{:04g}_skin{:g}_te{:g}/'.format(M, N, Ms, alpha*100, skin_on, trailing_edge_weight)
cases_subfolder = '/M{:g}N{:g}Ms{:g}_skin{:g}_te{:g}/'.format(M, N, Ms, skin_on, trailing_edge_weight)
batch_log = output_folder + 'batch_log_alpha{:04g}'.format(alpha*100)

try:
    os.mkdir(output_folder)
    print(f'Created folder: {output_folder}')
except FileExistsError:
    print(f'Folder exists: {output_folder}')
    pass

vf5 = postprocess_deformed_flutter.main(M=M, N=N, Ms=Ms, alpha=[alpha], skin=skin_on, te=trailing_edge_weight)
print('Flutter speeds', vf5)


with open('{:s}.txt'.format(batch_log), 'w') as f:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    f.write('SHARPy launch - START\n')
    f.write("date and time = %s\n\n" % dt_string)

    for i, u_inf in enumerate(vf5[::-1]):
        print('RUNNING SHARPY %f %f\n' % (alpha, u_inf))
        case_name = 'pazy_uinf{:04g}_alpha{:04g}'.format(u_inf*10, alpha*100)
        
        dt = 0.1/M/u_inf
        n_tsteps = int(end_time / dt)
        
        try:
            sharpy_output_nocontrol = generate_pazy_tseries(u_inf,
                                                            case_name,
                                                            output_folder=output_folder,
                                                            cases_subfolder=cases_subfolder,
                                                            M=M,
                                                            N=N,
                                                            Ms=Ms,
                                                            alpha=alpha,
                                                            gravity_on=gravity_on,
                                                            skin_on=skin_on,
                                                            trailing_edge_weight=trailing_edge_weight,
                                                            model_settings=pazy_settings,
                                                            num_cores=32,
                                                            end_time=end_time,
                                                            )
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            with open('{:s}.txt'.format(batch_log), 'a') as f:
                f.write('%s Ran case %i :::: u_inf = %f\n\n' % (dt_string, i, u_inf))
            
            # print(sharpy_output_nocontrol.structure.timestep_info[0].in_global_AFoR)
            # print(np.degrees(sharpy_output_nocontrol.structure.timestep_info[0].psi))
            # print(np.degrees(sharpy_output_nocontrol.structure.timestep_info[0].quat))
        
            time = np.linspace(0, dt*n_tsteps, n_tsteps)
            tip_pos_nocontrol = np.zeros((n_tsteps))
            tip_rot_nocontrol = np.zeros((n_tsteps))
            tip_pos_nocontrol[tip_pos_nocontrol==0] = None
            tip_rot_nocontrol[tip_rot_nocontrol==0] = None
            for it in range(n_tsteps):
                try:
                    tip_pos_nocontrol[it] = sharpy_output_nocontrol.structure.timestep_info[it].pos[-1, 2]
                    tip_rot_nocontrol[it] = sharpy_output_nocontrol.structure.timestep_info[it].psi[-1, -1, 1]
                except IndexError:
                    pass
            tip_rot_nocontrol = np.degrees(tip_rot_nocontrol)
            aileron = np.zeros((n_tsteps))
            aileron = np.degrees(aileron)
            
            # save nocontrol output to save some time
            file_out = np.vstack([time, tip_pos_nocontrol, tip_rot_nocontrol, aileron]).T  # time, h, theta, beta
            ustr = str(round(u_inf, 2)).replace('.', '_')
            filename = f'{output_folder}/tip_output_{ustr}.txt'
            np.savetxt(filename, file_out, delimiter=',')
            print(filename)

            # plotting, optional (plotted in conjunction with controlled output below)
            fig, ax = plt.subplots(3, figsize=(6, 6), dpi=200)

            ax[0].plot(time, tip_pos_nocontrol / (2*0.55), '-')
            ax[1].plot(time, tip_rot_nocontrol, '-')
            ax[2].plot(time, aileron, '-')

            ax[0].set_title(r'Pazy Wing, $V = {}$'.format(round(u_inf, 2)))
            ax[0].set_xlim(0, end_time)
            ax[1].set_xlim(0, end_time)
            ax[2].set_xlim(0, end_time)

            ax[0].grid()
            ax[1].grid()
            ax[2].grid()
            ax[2].set_xlabel(r"$t$ [s]")
            ax[0].set_ylabel(r"$\frac{h}{b}$")
            ax[1].set_ylabel(r'$\theta$ [deg]')
            ax[2].set_ylabel(r'$\beta$ [deg]')

            ustr = str(round(u_inf, 3)).replace('.', '_')
            #     plt.savefig('../Desktop/goland_vf_m{}n{}ms{}_nocontrol.png'.format(M, N, M_star_fact))
            plt.show()
        
        except AssertionError:
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            with open('./{:s}.txt'.format(batch_log), 'a') as f:
                f.write('%s ERROR RUNNING case %f\n\n' % (dt_string, u_inf))



Folder exists: output/pazy_M16N1Ms20_alpha0500_skin0_te0/
Unable to find eigenvalues at file /home/ianlamfar/sharpy_control/output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0590_alpha0500/pazy_uinf0590_alpha0500/stability/aeroelastic_eigenvalues.dat
No displacement files found at /home/ianlamfar/sharpy_control/output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0590_alpha0500/pazy_uinf0590_alpha0500/WriteVariablesTime/*
Unable to find eigenvalues at file /home/ianlamfar/sharpy_control/output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0530_alpha0500/pazy_uinf0530_alpha0500/stability/aeroelastic_eigenvalues.dat
No displacement files found at /home/ianlamfar/sharpy_control/output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0530_alpha0500/pazy_uinf0530_alpha0500/WriteVariablesTime/*
Unable to find eigenvalues at file /home/ianlamfar/sharpy_control/output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0550_alpha0500/pazy_uinf0550_al

TypeError: 'NoneType' object is not subscriptable

### Now run with controller

In [ ]:
import scipy.optimize as opt
import scipy.signal as sig
def amp(tseries):  # returns max average amplitude of wave-like time series
    _max = max(tseries)
    _min = min(tseries)
    avgmax = abs(_max - _min) / 2
    offset = np.mean([_max, _min])
    return avgmax, offset


def period(time, tseries):  # returns period of oscillation
    peaks = sig.find_peaks(tseries)[0]  # returns indices of peaks
    tpeak = time[peaks]
    intervals = np.diff(tpeak)
    return np.mean(intervals)


def peakfit(time, tseries):  # fits a decay function to the peaks
    peaks = sig.find_peaks(tseries)[0]  # returns indices of peaks
    tpeak = time[peaks]
    tspeak = tseries[peaks]
    def testfunc(x, a, b):  # test function, exponential decay here, x here corresponds to t
        return a * np.exp(b * x)
    popt, pcov = opt.curve_fit(testfunc, tpeak, tspeak)
    y = testfunc(time, *popt)
    return y, popt

In [1]:
import os
import sys
import numpy as np
sys.path.append('pazy/pazy_model/')
from generate_deformed_tseries import generate_pazy_tseries
import postprocess_deformed_flutter
flap_frac = 0.25
chord_frac = 0.25

# M = 16  # default 16
# N = 1  # default 1
# Ms = 20  # default 20
skin_on = False  # default False
trailing_edge_weight = False  # default False
alpha = 5
gravity_on = False
end_time = 0.1

M = 16
N = 1
Ms = 20

pazy_settings = {'num_elem': N,
                 'surface_m': M,
                'controlled': True,
                'controller_type': 'pde',
                'cs_deflection': [0, 0],
                'cs_span_frac': flap_frac,
                'cs_chord_frac': chord_frac,
                'skin_on': 'off',
                'discretisation_method': 'michigan',
                }


controller_id = {'aileron': 'ControlSurfacePdeController'}
controller_noise_settings = {'noise_mode': 'amplitude', 'max_percentage': 10, 'max_amplitude': np.radians(25),}
controller_lag_ms = 0
controller_p = 1
controller_i = 1


from datetime import datetime
# u_inf_vec = np.linspace(10, 90, 81)
# u_inf_vec = [83]

output_folder = './output/controlled/pazy_M{:g}N{:g}Ms{:g}_alpha{:04g}_skin{:g}_te{:g}_control/'.format(M, N, Ms, alpha*100, skin_on, trailing_edge_weight)
cases_subfolder = '/M{:g}N{:g}Ms{:g}_skin{:g}_te{:g}_control/'.format(M, N, Ms, skin_on, trailing_edge_weight)
batch_log = output_folder + 'batch_log_alpha{:04g}_control'.format(alpha*100)

try:
    os.makedirs(output_folder)
    print(f'Created folder: {output_folder}')
except FileExistsError:
    print(f'Folder exists: {output_folder}')
    pass

vf5 = postprocess_deformed_flutter.main(M=M, N=N, Ms=Ms, alpha=[alpha], skin=skin_on, te=trailing_edge_weight)
print('Flutter speeds', vf5)


with open('{:s}.txt'.format(batch_log), 'w') as f:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    f.write('SHARPy launch - START\n')
    f.write("date and time = %s\n\n" % dt_string)

    for i, u_inf in enumerate(vf5[::-1]):
        print('RUNNING SHARPY %f %f\n' % (alpha, u_inf))
        case_name = 'pazy_uinf{:04g}_alpha{:04g}'.format(u_inf*10, alpha*100)
        
        dt = 0.1/M/u_inf
        # n_tsteps = int(end_time / dt)
        n_tsteps = 5
        
        controller_settings = {'aileron': {'P': controller_p,
                                    'I': controller_i,
                                    'dt': dt,
                                    'controlled_surfaces': [0, 1],
                                    'write_controller_log': True,
                                    'input_type': 'tip rotation',
                                    'time_history_input_file': f'{output_folder}controller/input.txt',
                                    'controller_log_route': f'{output_folder}controller/',
                                    'controller_noise': False,
                                    'controller_noise_settings': controller_noise_settings,
                                    'controller_lag': int(controller_lag_ms/1000/dt),
                                    'output_limit': np.radians(25),  # IN RADIANS, -1 if no limit
                                    }
                                }
        
        try:
            sharpy_output_control = generate_pazy_tseries(u_inf,
                                                            case_name,
                                                            output_folder=output_folder,
                                                            cases_subfolder=cases_subfolder,
                                                            M=M,
                                                            N=N,
                                                            Ms=Ms,
                                                            alpha=alpha,
                                                            gravity_on=gravity_on,
                                                            skin_on=skin_on,
                                                            trailing_edge_weight=trailing_edge_weight,
                                                            model_settings=pazy_settings,
                                                            num_cores=32,
                                                            end_time=end_time,
                                                            controller_id=controller_id,
                                                            controller_settings=controller_settings,
                                                            )
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            with open('{:s}.txt'.format(batch_log), 'a') as f:
                f.write('%s Ran case %i :::: u_inf = %f\n\n' % (dt_string, i, u_inf))
            
            data = np.genfromtxt(f'{output_folder}controller/aileron_log.csv', delimiter=',')
            tstep = data[:, 0]
            tctrl = data[:, 1]
            desired = np.degrees(data[:, 2])
            current = np.degrees(data[:, 3])
            pctrl = np.degrees(data[:, 4])
            ictrl = np.degrees(data[:, 5])
            dctrl = np.degrees(data[:, 6])
            nctrl = np.degrees(data[:, 7])
            cap_ctrl = data[:, 8]
            rctrl = np.degrees(data[:, 9])
            ctrl = np.degrees(data[:, -1])
            
            ustr = str(round(u_inf, 2)).replace('.', '_')
            filename = output_folder + f'tip_output{ustr}.txt'
            a = np.loadtxt(filename, delimiter=',')
            t_nocontrol = a[:, 0]
            tip_pos_nocontrol = a[:, 1]
            tip_rot_nocontrol = a[:, 2]

            time = np.linspace(0, dt*n_tsteps, n_tsteps)
            tip_pos = np.zeros((n_tsteps))
            tip_rot = np.zeros((n_tsteps))
            aileron = np.zeros((n_tsteps))
            tip_pos[tip_pos==0] = None
            tip_rot[tip_rot==0] = None
            aileron[aileron==0] = None
            for it in range(0, len(time)):  # fill values up to latest timestep, does not necessarily be all timesteps if iteration ended early due to errors
                try:
                    tip_pos[it] = sharpy_output_control.structure.timestep_info[it].pos[-1, 2]
                    tip_rot[it] = sharpy_output_control.structure.timestep_info[it].psi[-1, -1, 1]
                except IndexError:
                    pass
                try:
                    aileron[it] = sharpy_output_control.aero.timestep_info[it].control_surface_deflection[0]
                except:
                    pass

            tip_rot = np.degrees(tip_rot)
            aileron = np.degrees(aileron)

            '''print(rctrl)
            print(ctrl)
            print(aileron)
            print(ctrl.shape)
            print(aileron.shape)'''

            # amplitude and offset
            amp_nocontrol, off_nocontrol = amp(tip_rot_nocontrol)
            amp_control, off_control = amp(tip_rot)
            amp_aileron, off_aileron = amp(ctrl)
            amp_raw, off_raw = amp(rctrl)

            # identifying peaks
            tip_rot_peaks = sig.find_peaks(tip_rot)[0]
            # tip_rot_nocontrol_peaks = sig.find_peaks(tip_rot_nocontrol)[0]

            '''tipfit, params = peakfit(time, np.radians(tip_rot))
            tipfit = np.degrees(tipfit)
            osc_period = period(time, tip_rot)
            print(params)
            '''
            # Plotting
            fig, ax = plt.subplots(3, figsize=(9, 9), dpi=200)

            ax[0].plot(t_nocontrol, tip_pos_nocontrol / ws.b_ref, '-', label='No PID')
            ax[0].plot(time, tip_pos / ws.b_ref, '-', label='With PID')

            ax[1].plot(t_nocontrol, tip_rot_nocontrol, '-', label=r'No PID $(A = {:.2f}^{}, D = {:.2f}^{})$'.format(amp_nocontrol, r'\circ', off_nocontrol, r'\circ'))
            # ax[1].plot(time, tip_rot, '-', label='With PID\n'+r'$A = {:.2f}$, $\theta = {:.3f} \exp ({:.3f}t)$'.format(amp(tip_rot)[0], params[0], params[1]))
            ax[1].plot(time, tip_rot, '-', label=r'With PID ($A = {:.2f}^{}, D = {:.2f}^{}$)'.format(amp_control, r'\circ', off_control, r'\circ'))
            # ax[1].plot(time[tip_rot_nocontrol_peaks], tip_rot_nocontrol[tip_rot_nocontrol_peaks], 'x')
            # ax[1].plot(time[tip_rot_peaks], tip_rot[tip_rot_peaks], 'x')
            # ax[1].plot(time, tipfit, alpha=0.5, linewidth=1)

            if (rctrl == ctrl).all():
                ax[2].plot(tctrl, ctrl, '-', label='Sum\n'+r'$(A = {:.2f}^{}, D = {:.2f}^{})$'.format(amp_aileron, r'\circ', off_aileron, r'\circ'))
            else:
                ax[2].plot(tctrl, rctrl, '-', label='Sum (raw) '+r'$(A = {:.2f}^{}, D = {:.2f}^{})$'.format(amp_raw, r'\circ', off_raw, r'\circ'))
                ax[2].plot(tctrl, ctrl, '-', label='Sum (capped) '+r'$(A = {:.2f}^{}, D = {:.2f}^{})$'.format(amp_aileron, r'\circ', off_aileron, r'\circ'))
            ax[2].plot(tctrl, pctrl, '--', label=r'P$={}$'.format(P), alpha=0.5, linewidth=1)
            ax[2].plot(tctrl, ictrl, '--', label=r'I$={}$'.format(I), alpha=0.5, linewidth=1)
            ax[2].plot(tctrl, dctrl, '--', label=r'D$={}$'.format(D), alpha=0.5, linewidth=1)
            ax[2].set_xlim(0, end_time)
            #ax[2].set_ylim(-1, 1)
            #ax[1].plot(time, np.degrees(current))
            #ax[2].plot(time, np.degrees(aileron))

            # ax2t = ax[2].twinx()
            # ax2t.plot(tctrl, cap_ctrl, 'C4', label='Output capping', alpha=0.5)
            # ax2t.set_xlim(0, end_time)
            # ax2t.set_ylim(-1.1, 1.1)

            # prepare noise string
            if noise_settings['noise_mode'] == 'percentage':
                noisestr = '{}'.format(noise_settings['max_percentage']) + r'%'
            elif noise_settings['noise_mode'] == 'amplitude':
                noisestr = '{}'.format(np.degrees(noise_settings['max_amplitude'])) + '$^{\circ}$'

            if noise and lag == 0:
                ax[0].set_title(r'Goland Wing, $V = V_F = {:.2f}$ '.format(vf) + 'm s$^{-1}$' + ', with max. {} controller noise'.format(noisestr))
            elif noise and lag > 0:
                ax[0].set_title(r'Goland Wing, $V = V_F = {:.2f}$ ' + 'm s$^{-1}$' + ', with controller noise, controller lag = {} ms'.format(round(lag_ms)))
            elif noise is False and lag > 0:
                ax[0].set_title(r'Goland Wing, $V = V_F = {:.2f}$ '.format(vf) + 'm s$^{-1}$' + ', controller lag = {} ms'.format(round(lag_ms)))
            else:
                ax[0].set_title(r'Goland Wing, $V = V_F = {:.2f}$'.format(vf)+' m s$^{-1}$')
            ax[0].set_xlim(0, end_time)
            ax[1].set_xlim(0, end_time)
            ax[2].set_xlim(0, end_time)

            # ax[1].set_ylim(-1, 1)


            ax[0].grid()
            ax[1].grid()
            ax[2].grid()
            ax[2].set_xlabel(r"$t$ [s]")
            ax[0].set_ylabel(r"$\frac{h}{b}$")
            ax[1].set_ylabel(r'$\theta$ [deg]')
            ax[2].set_ylabel(r'$\beta$ [deg]')
            ax[0].legend()
            ax[1].legend()
            ax[2].legend(loc=0, fontsize=8)

            # prepare strings for saving files
            ustr = str(round(vf, 3)).replace('.', '_')
            pstr = str(P).replace('.', '_')
            istr = str(I).replace('.', '_')
            dstr = str(D).replace('.', '_')
            lstr = str(round(lag_ms)).replace('.', '_')
            if noise_settings['noise_mode'] == 'percentage':
                nstr = ('{}pct'.format(noise_settings['max_percentage'])).replace('.', '_')
            elif noise_settings['noise_mode'] == 'amplitude':
                nstr = ('{}deg'.format(np.degrees(noise_settings['max_amplitude']))).replace('.', '_')

            # if noise and lag == 0:
            #     plt.savefig('../Desktop/flutter_control/goland_vf_m{}n{}ms{}_control_p{}i{}d{}_noise{}.png'.format(M, N, M_star_fact, pstr, istr, dstr, nstr))
            # elif noise and lag > 0:
            #     plt.savefig('../Desktop/flutter_control/goland_vf_m{}n{}ms{}_control_p{}i{}d{}_noise{}_lag{}ms.png'.format(M, N, M_star_fact, pstr, istr, dstr, nstr, lstr))
            # elif noise is False and lag > 0:
            #     plt.savefig('../Desktop/flutter_control/goland_vf_m{}n{}ms{}_control_p{}i{}d{}_lag{}ms.png'.format(M, N, M_star_fact, pstr, istr, dstr, lstr))
            # else:
            #     plt.savefig('../Desktop/flutter_control/goland_vf_m{}n{}ms{}_control_p{}i{}d{}.png'.format(M, N, M_star_fact, pstr, istr, dstr))
            plt.show()

        
            time = np.linspace(0, dt*n_tsteps, n_tsteps)
            tip_pos_control = np.zeros((n_tsteps))
            tip_rot_control = np.zeros((n_tsteps))
            tip_pos_control[tip_pos_control==0] = None
            tip_rot_control[tip_rot_control==0] = None
            for it in range(n_tsteps):
                try:
                    tip_pos_control[it] = sharpy_output_control.structure.timestep_info[it].pos[-1, 2]
                    tip_rot_control[it] = sharpy_output_control.structure.timestep_info[it].psi[-1, -1, 1]
                except IndexError:
                    pass
            tip_rot_nocontrol = np.degrees(tip_rot_nocontrol)
            aileron = np.zeros((n_tsteps))
            aileron = np.degrees(aileron)
            
            # save nocontrol output to save some time
            file_out = np.vstack([time, tip_pos_control, tip_rot_control, aileron]).T  # time, h, theta, beta
            ustr = str(round(u_inf, 2)).replace('.', '_')
            filename = f'{output_folder}/tip_output_{ustr}.txt'
            np.savetxt(filename, file_out, delimiter=',')
            print(filename)

            # plotting, optional (plotted in conjunction with controlled output below)
            fig, ax = plt.subplots(3, figsize=(6, 6), dpi=200)

            ax[0].plot(time, tip_pos_nocontrol / (2*0.55), '-')
            ax[1].plot(time, tip_rot_nocontrol, '-')
            ax[2].plot(time, aileron, '-')

            ax[0].set_title(r'Pazy Wing, $V = {}$'.format(round(u_inf, 2)))
            ax[0].set_xlim(0, end_time)
            ax[1].set_xlim(0, end_time)
            ax[2].set_xlim(0, end_time)

            ax[0].grid()
            ax[1].grid()
            ax[2].grid()
            ax[2].set_xlabel(r"$t$ [s]")
            ax[0].set_ylabel(r"$\frac{h}{b}$")
            ax[1].set_ylabel(r'$\theta$ [deg]')
            ax[2].set_ylabel(r'$\beta$ [deg]')

            ustr = str(round(u_inf, 3)).replace('.', '_')
            #     plt.savefig('../Desktop/goland_vf_m{}n{}ms{}_nocontrol.png'.format(M, N, M_star_fact))
            plt.show()
        
        except AssertionError:
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            with open('./{:s}.txt'.format(batch_log), 'a') as f:
                f.write('%s ERROR RUNNING case %f\n\n' % (dt_string, u_inf))



Folder exists: ./output/controlled/pazy_M16N1Ms20_alpha0500_skin0_te0_control/
Unable to find source case .pmor.sharpy at ./output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0450_alpha0500
Loaded 35 cases from ['./output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0370_alpha0500', './output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0300_alpha0500', './output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0160_alpha0500', './output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0310_alpha0500', './output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0390_alpha0500', './output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0420_alpha0500', './output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0200_alpha0500', './output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0230_alpha0500', './output/pazy/output/pazy_M16N1Ms20_alpha0500_skin0_te0/pazy_uinf0210_alpha0500', './output/pazy/output/pazy_M1

FileNotFoundError: [Errno 2] No such file or directory: './output/controlled/pazy_M16N1Ms20_alpha0500_skin0_te0_control/controller/aileron_log.csv'